In [ ]:
impute = ColumnTransformer([('num_impute', SimpleImputer(strategy='mean'), ['Age']), 
                            ('cab_impute', SimpleImputer(strategy='constant'), ['Cabin']), 
                            ('emb_impute', SimpleImputer(strategy='most_frequent'), ['Embarked'])], 
                          remainder='passthrough' )#, 
                      # ('model', LogisticRegression()),])

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)

param_grid = [{ #regular grid search took about 1 ahour with KNNImputer
            'model__n_estimators': [10, 100, 1000],
            'model__learning_rate': [0.001, 0.01, 0.1],
            'model__subsample': [0.5, 0.7, 1.0],
            'model__max_depth': [3, 7, 9],
    
#             'imputer__imputer__strategy': ['mean', 'median', 'constant', 'most_frequent']
            'imputer__num_imputer__strategy': ['mean'], # , median, constant, most_frequent
            'imputer__cab_imputer__strategy': ['constant', 'most_frequent'], # constant, most_frequent
            'imputer__cab_imputer__strategy': ['most_frequent']
              }]

In [ ]:
#pipeline = Pipeline([('clean_d', cleanData)], verbose=True)

# classifier = Pipeline([('imputer', impute), 
#                      ('embarked', codeEmbarked_toPipeline_trans),
#                      ("titles", getTitles_trans),
#                      ("fare", fare_trans),
#                      ('age', age_trans),
#                      ('family', familySize_trans), 
#                      ('oneHot', sexTitle_to_OneHot_trans), 
#                      ('model', model)])

cleaner = Pipeline([ 
                     ('imputer', impute), 
                     ('to_df', toDF_trans)
                     ('embarked', codeEmbarked_trans), # para clean: codeEmbarked_trans
                     ("titles", getTitles_trans),
#                     ("fare", fare_trans),
#                      ('age', age_trans),
#                      ('family', familySize_trans), 
#                      ('oneHot', sexTitle_to_OneHot_trans) 
                   ])

In [ ]:
def to_df(a):
    
    
    df = pd.DataFrame(a.copy(), columns=['Age', 'Cabin', 'Embarked', 'Pclass', 
                                         'Name', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Fare'])
    
    return df


def fare(df):
    #s['FareBand'] = pd.qcut(s['Fare'], 4)
    
    s = df.copy()
    s.loc[ s['Fare'] <= 7.91, 'Fare'] = 0
    s.loc[(s['Fare'] > 7.91) & (s['Fare'] <= 14.454), 'Fare'] = 1
    s.loc[(s['Fare'] > 14.454) & (s['Fare'] <= 31), 'Fare']   = 2
    s.loc[ s['Fare'] > 31, 'Fare'] = 3

    #Cambiar tipo de columna
    s['Fare'] = s['Fare'].astype(int)
    
    return s

def age(df): #if it uses 2 times it will set all values to 1.0
            #recive a copy
            #does not touch nan values
            
    s = df.copy()
    s.loc[(s.Age <= 13), 'Age'] = 1
    s.loc[(s.Age > 13) & (s.Age <= 18), 'Age'] = 2
    s.loc[(s.Age > 18) & (s.Age <= 30), 'Age'] = 3
    s.loc[(s.Age > 30) & (s.Age <= 50), 'Age'] = 4
    s.loc[(s.Age > 50), 'Age'] = 5
    
    return s

def familySize(df):
    
    s = df.copy()
    s['FamilySize'] = s['SibSp'] +  s['Parch'] + 1
    
    s['FamilySize'] = s['FamilySize'].astype(int)
    #s['Survived'] = s['Survived'].astype(int)
    
    s['IsAlone'] = 0
    s.loc[s['FamilySize'] == 1, 'IsAlone'] = 1
    
    return s.drop(columns=['SibSp', 'Parch', 'FamilySize']).copy()


def getTitles(df):
    #str.extract(' ([A-Za-z]+)\.')
    #print(type(s))
    
    s = df.copy()
    #print(s.sample(7))
    #assert 0
    s['Title'] = s['Name'].str.extract(' ([A-Za-z]+)\.')
    s['Title'] = s['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr',                                       
                                    'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
    s['Title'] = s['Title'].replace(['Mlle', 'Ms'], 'Miss')
    s['Title'] = s['Title'].replace(['Mme'], 'Mrs')
    
    #print(s.sample(7))
    #assert 0
    return s.drop(columns='Name').copy()

def codeEmbarked(df): #Usado en Cleaner nada mas
    '''It recive a numpy array so the first step is turning in to a data frame'''
    
#     s = pd.DataFrame(df.copy(), columns=['Age', 'Cabin', 'Embarked', 'Pclass', 
#                                          'Name', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Fare'])
    s = df.copy()
    s['Embarked'] = s['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2})#.astype('int') se lo quito para que pueda dejar nan
    
    return s

def codeEmbarked_toPipeline(df): #usado con imputer en pipeline
    '''It recive a numpy array so the first step is turning in to a data frame'''
    print(df)
    s = pd.DataFrame(df.copy(), columns=['Age', 'Cabin', 'Embarked', 'Pclass', 
                                         'Name', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Fare'])
    
    print(s[60:63]) 
#     s = df.copy()              # map hace nan los valores que no encuentra mach
    s['Embarked'] = s['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2})#.astype('int') se lo quito para que pueda dejar nan
    
    return s

def sexTitle_to_OneHot(df):
    '''This version works when you firts split the data set into X, y dropping 'Survived' 
       from X
    '''
    encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    
    
    #d_clean.drop(columns=['Ticket', 'Cabin'], inplace=True)
    s = df.drop(columns=['Ticket', 'Cabin']).copy()
    #print(s.head(16))
    #print(s.columns)
    #s.columns
    
    # ['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
    
    cat_features = ['Sex', 'Title']
    
    #res_features = ['Age', 'Embarked', 'Pclass', 'Fare', 'IsAlone']
    
    # Pclass  Age  Fare  Embarked  IsAlone
    res_features = ['Pclass', 'Age', 'Fare', 'Embarked', 'IsAlone']
    
    transformer = ColumnTransformer([('cat', encoder, cat_features)], remainder='passthrough')
    
    transformer.fit(s)
    
    tranformed_names = transformer.named_transformers_['cat'].get_feature_names_out(cat_features)
    #tranformed_names
    
    new_names = np.append(tranformed_names, res_features)
    
    #print(new_names)
    #assert 0
    
    return pd.DataFrame(transformer.transform(s), columns=new_names)



# Para prepara el df para age y embarked prediction

def Age_Embarked(df):

    age_X_te = df.loc[df.Age.isna()].copy()
    embarked_X_te = df.loc[df.Embarked.isna()].copy()

    embarked_X_te.drop(columns=['Embarked'], inplace=True)
    age_X_te.drop(columns=['Age'], inplace=True)


    age_X_tr = df.dropna(subset=['Age', 'Embarked']).copy()
    age_y_tr = age_X_tr.Age


    embarked_X_tr = df.dropna(subset=['Age', 'Embarked']).copy()
    embarked_y_tr = embarked_X_tr.Embarked

    # to leave the df with all columns
    # age_X_tr.drop(columns=['Age'], inplace=True)
    # embarked_X_tr.drop(columns=['Embarked'], inplace=True)
    
    return (age_X_te, embarked_X_te, age_X_tr, age_y_tr, embarked_X_tr, embarked_y_tr)

def get_x_y(df, y):
    #resives an copy of a df and return X_tr, y_tr
    X_tr = df.drop(columns=[y])
    y_tr = df[y]
    
    return X_tr, y_tr


#temp01 = temp01.drop(columns=['Ticket', 'Cabin'])

def cleanData(df):
    
    d_clean = df.copy()
    
    #d_clean = getTitles(df)
    
    #print('dataCleanAdentro')
    #print(d_clean.columns)
    #assert 0
    d_clean = fare(d_clean)
    d_clean = age(d_clean)
    d_clean = familySize(d_clean) #** seem all good so far.
    #print(d_clean.columns)
    #assert 0

    #OneHot aplied
    d_clean.drop(columns=['Ticket', 'Cabin'], inplace=True)

    d_clean = sexTitle_to_OneHot(d_clean)
    
    d_clean = codeEmbarked(d_clean)
    
    #d_clean['Embarked'] = d_clean['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2})#.astype(int)
        
    return d_clean

def reset_index(df):
    df = df.reset_index(inplace = False)
    return df

codeEmbarked_trans = FunctionTransformer(codeEmbarked, validate=False)
getTitles_trans = FunctionTransformer(getTitles, validate=False)
sexTitle_to_OneHot_trans = FunctionTransformer(sexTitle_to_OneHot, validate=False)
fare_trans = FunctionTransformer(fare, validate=False)
age_trans = FunctionTransformer(age, validate=False)
familySize_trans = FunctionTransformer(familySize, validate=False)
toDF_trans = FunctionTransformer(to_df, validate=False)

codeEmbarked_toPipeline_trans = FunctionTransformer(codeEmbarked_toPipeline, validate=False)